# Manutenzione preventiva - DoS Attacks

In [1]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import warnings
import functions_notebook_new as fn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC


warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
### costanti
# clean captures
clean_capture = 'captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json'

# ping flooding captures 1 min
pf_capture_1 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS1m-0,5h_1.json'
pf_capture_2 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS5m-0,5h_1.json'
pf_capture_3 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'

levels_of_interest = ['frame', 'eth', 'tcp', 'arp', 'udp', 'modbus', 'icmp', 'ip']

### variabili

In [3]:
### estrazione dati
# packets clean capture
packets_cc = fn.load_packets_from_file(clean_capture)

# packets attack captures (es. packets ping flooding attack 1m per cattura di 0.5 = pkt_pf_1m_05h)
pkt_pf_1m_05h = fn.load_packets_from_file(pf_capture_1)
pkt_pf_5m_05h = fn.load_packets_from_file(pf_capture_2)
pkt_pf_15m_05h = fn.load_packets_from_file(pf_capture_3)

In [4]:
### features
# clean features
features_cc = fn.extract_features_from_packets(packets_cc, levels_of_interest)

# ping flooding attack features
features_pf_1m_05h = fn.extract_features_from_packets(pkt_pf_1m_05h, levels_of_interest)
features_pf_5m_05h = fn.extract_features_from_packets(pkt_pf_5m_05h, levels_of_interest)
features_pf_15m_05h =fn.extract_features_from_packets(pkt_pf_15m_05h, levels_of_interest)

In [5]:
### Dataframes
# df clean capture
df_clean = pd.DataFrame(features_cc)

# df ping flooding attacks captures
df_pf_1m_05h = pd.DataFrame(features_pf_1m_05h)
df_pf_5m_05h = pd.DataFrame(features_pf_5m_05h)
df_pf_15m_05h = pd.DataFrame(features_pf_15m_05h)


pd_concatenation = pd.concat([df_clean, df_pf_1m_05h, df_pf_5m_05h, df_pf_15m_05h])

In [6]:
pd_concatenation.isna().sum().to_excel('check_nulls_number.xlsx')

In [7]:
df_colonne_senza_nan = pd_concatenation.loc[:, pd_concatenation.isna().sum() == 0].copy()

In [8]:
len(df_colonne_senza_nan.select_dtypes(include=['object']).columns)

60

In [9]:
df_colonne_senza_nan.select_dtypes(include=['object']).iloc[:10,50:60]

,eth.eth.src_tree.eth.addr.oui_resolved_raw[0],eth.eth.src_tree.eth.addr.oui_resolved,eth.eth.src_tree.eth.src.lg_raw[0],eth.eth.src_tree.eth.src.lg,eth.eth.src_tree.eth.lg_raw[0],eth.eth.src_tree.eth.lg,eth.eth.src_tree.eth.src.ig_raw[0],eth.eth.src_tree.eth.src.ig,eth.eth.src_tree.eth.ig_raw[0],eth.eth.src_tree.eth.ig
0,000c299d9e9e,"VMware, Inc.",0,0,0,0,0,0,0,0
1,d07e288e40b3,Hewlett Packard,0,0,0,0,0,0,0,0
2,000c299d9e9e,"VMware, Inc.",0,0,0,0,0,0,0,0
3,0080f409513b,Telemecanique Electrique,0,0,0,0,0,0,0,0
4,000c299d9e9e,"VMware, Inc.",0,0,0,0,0,0,0,0
5,000c299d9e9e,"VMware, Inc.",0,0,0,0,0,0,0,0
6,0080f409513b,Telemecanique Electrique,0,0,0,0,0,0,0,0
7,485b39644079,ASUSTek COMPUTER INC.,0,0,0,0,0,0,0,0
8,0080f409513b,Telemecanique Electrique,0,0,0,0,0,0,0,0
9,0080f409513b,Telemecanique Electrique,0,0,0,0,0,0,0,0


### Feature tipo intero
frame.frame.encap_type
frame.frame.number
frame.frame.len
eth.eth.dst_tree.eth.dst.oui	
frame.frame.cap_len	frame.frame.marked	frame.frame.ignored
eth.eth.dst_tree.eth.dst.lg_raw[0]	
eth.eth.dst_tree.eth.addr.oui
eth.eth.dst_tree.eth.dst.lg
eth.eth.dst_tree.eth.lg_raw[0]	eth.eth.dst_tree.eth.lg	eth.eth.dst_tree.eth.dst.ig_raw[0]	eth.eth.dst_tree.eth.dst.ig	eth.eth.dst_tree.eth.ig_raw[0]	eth.eth.dst_tree.eth.ig
eth.eth.src
eth.eth.src_tree.eth.src.oui	
eth.eth.src_tree.eth.addr.oui
eth.eth.src_tree.eth.src.lg_raw[0]	eth.eth.src_tree.eth.src.lg	eth.eth.src_tree.eth.lg_raw[0]	eth.eth.src_tree.eth.lg	eth.eth.src_tree.eth.src.ig_raw[0]	eth.eth.src_tree.eth.src.ig	eth.eth.src_tree.eth.ig_raw[0]	eth.eth.src_tree.eth.ig
#### ESADECIMALE
eth.eth.dst_raw[0]	 eth.eth.dst_tree.eth.dst_resolved_raw[0] eth.eth.dst_tree.eth.dst.oui_raw[0] eth.eth.dst_tree.eth.addr_raw[0] eth.eth.dst_tree.eth.addr.oui_raw[0]
eth.eth.src_raw[0]
eth.eth.src_tree.eth.src_resolved_raw[0]	
eth.eth.src_tree.eth.src.oui_raw[0]
eth.eth.src_tree.eth.src.oui_resolved_raw[0]
eth.eth.src_tree.eth.addr_raw[0]
eth.eth.src_tree.eth.addr
eth.eth.src_tree.eth.addr_resolved_raw[0]
eth.eth.src_tree.eth.addr.oui_raw[0]
eth.eth.src_tree.eth.addr.oui_resolved_raw[0]				
#### MAC ADDRESS	
eth.eth.dst mac address eth.eth.dst_tree.eth.addr eth.eth.dst_tree.eth.addr_resolved_raw[0] 	 (da rimuovere : e conversione in esadecimale)

### Feature tipo float
frame.frame.time_epoch
frame.frame.offset_shift
frame.frame.time_delta
frame.frame.time_delta_displayed
frame.frame.time_relative
### Feature tipo string
frame.frame.protocols	da ricavare i protocolli contenuti all'interno del frame
#### DA SCARTARE
'frame.frame.coloring_rule.name'
'frame.frame.coloring_rule.string'
'eth.eth.dst_tree.eth.dst_resolved'
'eth.eth.dst_tree.eth.addr_resolved'
'eth.eth.src_tree.eth.src_resolved'
'eth.eth.src_tree.eth.src.oui_resolved'	
'eth.eth.src_tree.eth.addr_resolved'
'eth.eth.src_tree.eth.addr.oui_resolved'

In [10]:
df_colonne_senza_nan.drop(columns=['frame.frame.coloring_rule.name','frame.frame.coloring_rule.string', 'eth.eth.dst_tree.eth.dst_resolved', 'eth.eth.dst_tree.eth.addr_resolved', 'eth.eth.src_tree.eth.src_resolved', 'eth.eth.src_tree.eth.src.oui_resolved', 'eth.eth.src_tree.eth.addr_resolved', 'eth.eth.src_tree.eth.addr.oui_resolved'], inplace=True)

In [11]:
len(df_colonne_senza_nan)

340626

In [12]:
fn.df_convert(df_colonne_senza_nan)
df_colonne_senza_nan

frame.frame.encap_type
frame.frame.number
frame.frame.len
eth.eth.dst_tree.eth.dst.oui
frame.frame.cap_len
frame.frame.marked
frame.frame.ignored
eth.eth.dst_tree.eth.dst.lg_raw[0]
eth.eth.dst_tree.eth.addr.oui
eth.eth.dst_tree.eth.dst.lg
eth.eth.dst_tree.eth.lg_raw[0]
eth.eth.dst_tree.eth.lg
eth.eth.dst_tree.eth.dst.ig_raw[0]
eth.eth.dst_tree.eth.dst.ig
eth.eth.dst_tree.eth.ig_raw[0]
eth.eth.dst_tree.eth.ig
eth.eth.src_tree.eth.src.oui
eth.eth.src_tree.eth.addr.oui
eth.eth.src_tree.eth.src.lg_raw[0]
eth.eth.src_tree.eth.src.lg
eth.eth.src_tree.eth.lg_raw[0]
eth.eth.src_tree.eth.lg
eth.eth.src_tree.eth.src.ig_raw[0]
eth.eth.src_tree.eth.src.ig
eth.eth.src_tree.eth.ig_raw[0]
eth.eth.src_tree.eth.ig


,frame.frame.encap_type,frame.frame.time,frame.frame.time_utc,frame.frame.time_epoch,frame.frame.offset_shift,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len,...,eth.eth.src_tree.eth.src.ig_raw[2],eth.eth.src_tree.eth.src.ig_raw[3],eth.eth.src_tree.eth.src.ig_raw[4],eth.eth.src_tree.eth.src.ig,eth.eth.src_tree.eth.ig_raw[0],eth.eth.src_tree.eth.ig_raw[1],eth.eth.src_tree.eth.ig_raw[2],eth.eth.src_tree.eth.ig_raw[3],eth.eth.src_tree.eth.ig_raw[4],eth.eth.src_tree.eth.ig
0,1,"Aug 23, 2018 19:40:48.376131000 ora legale Eur...","Aug 23, 2018 17:40:48.376131000 UTC",1.535046e+09,0.0,0.000000,0.000000,0.000000,1,60,...,3,65536,2,0,0,6,3,65536,2,0
1,1,"Aug 23, 2018 19:40:48.397071000 ora legale Eur...","Aug 23, 2018 17:40:48.397071000 UTC",1.535046e+09,0.0,0.020940,0.020940,0.020940,2,64,...,3,65536,2,0,0,6,3,65536,2,0
2,1,"Aug 23, 2018 19:40:48.470440000 ora legale Eur...","Aug 23, 2018 17:40:48.470440000 UTC",1.535046e+09,0.0,0.073369,0.073369,0.094309,3,66,...,3,65536,2,0,0,6,3,65536,2,0
3,1,"Aug 23, 2018 19:40:48.473558000 ora legale Eur...","Aug 23, 2018 17:40:48.473558000 UTC",1.535046e+09,0.0,0.003118,0.003118,0.097427,4,85,...,3,65536,2,0,0,6,3,65536,2,0
4,1,"Aug 23, 2018 19:40:48.688103000 ora legale Eur...","Aug 23, 2018 17:40:48.688103000 UTC",1.535046e+09,0.0,0.214545,0.214545,0.311972,5,60,...,3,65536,2,0,0,6,3,65536,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197773,1,"May 21, 2018 13:42:36.345657000 ora legale Eur...","May 21, 2018 11:42:36.345657000 UTC",1.526903e+09,0.0,0.001220,0.001220,1798.996934,197774,60,...,3,65536,2,0,0,6,3,65536,2,0
197774,1,"May 21, 2018 13:42:36.345673000 ora legale Eur...","May 21, 2018 11:42:36.345673000 UTC",1.526903e+09,0.0,0.000016,0.000016,1798.996950,197775,60,...,3,65536,2,0,0,6,3,65536,2,0
197775,1,"May 21, 2018 13:42:36.345854000 ora legale Eur...","May 21, 2018 11:42:36.345854000 UTC",1.526903e+09,0.0,0.000181,0.000181,1798.997131,197776,60,...,3,65536,2,0,0,6,3,65536,2,0
197776,1,"May 21, 2018 13:42:36.354401000 ora legale Eur...","May 21, 2018 11:42:36.354401000 UTC",1.526903e+09,0.0,0.008547,0.008547,1799.005678,197777,60,...,3,65536,2,0,0,6,3,65536,2,0


In [13]:
for column, dtype in df_colonne_senza_nan.dtypes.items():
    print(f"Column: {column}, Data Type: {dtype}")

Column: frame.frame.encap_type, Data Type: int64
Column: frame.frame.time, Data Type: object
Column: frame.frame.time_utc, Data Type: object
Column: frame.frame.time_epoch, Data Type: float64
Column: frame.frame.offset_shift, Data Type: float64
Column: frame.frame.time_delta, Data Type: float64
Column: frame.frame.time_delta_displayed, Data Type: float64
Column: frame.frame.time_relative, Data Type: float64
Column: frame.frame.number, Data Type: int64
Column: frame.frame.len, Data Type: int64
Column: frame.frame.cap_len, Data Type: int64
Column: frame.frame.marked, Data Type: int64
Column: frame.frame.ignored, Data Type: int64
Column: frame.frame.protocols, Data Type: object
Column: eth.eth.dst_raw[0], Data Type: int64
Column: eth.eth.dst_raw[1], Data Type: int64
Column: eth.eth.dst_raw[2], Data Type: int64
Column: eth.eth.dst_raw[3], Data Type: int64
Column: eth.eth.dst_raw[4], Data Type: int64
Column: eth.eth.dst, Data Type: int64
Column: eth.eth.dst_tree.eth.dst_resolved_raw[0], Dat